In [37]:
import numpy as np
import pandas as pd

In [38]:
insta = pd.read_csv('installments_payments.csv')
insta

,SK_ID_PREV,SK_ID_CURR,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT
0,1054186,161674,1.0,6,-1180.0,-1187.0,6948.360,6948.360
1,1330831,151639,0.0,34,-2156.0,-2156.0,1716.525,1716.525
2,2085231,193053,2.0,1,-63.0,-63.0,25425.000,25425.000
3,2452527,199697,1.0,3,-2418.0,-2426.0,24350.130,24350.130
4,2714724,167756,1.0,2,-1383.0,-1366.0,2165.040,2160.585
...,...,...,...,...,...,...,...,...
13605396,2186857,428057,0.0,66,-1624.0,NaN,67.500,NaN
13605397,1310347,414406,0.0,47,-1539.0,NaN,67.500,NaN
13605398,1308766,402199,0.0,43,-7.0,NaN,43737.435,NaN
13605399,1062206,409297,0.0,43,-1986.0,NaN,67.500,NaN


In [39]:
insta = insta.sort_values('DAYS_INSTALMENT')

In [40]:
# 'DAYS_INSTALMENT': days before credit card supposed to be paid, 'DAYS_ENTRY_PAYMENT': days that amount was acutually paid
insta['insta_delinquency'] = np.where(insta['DAYS_INSTALMENT'] >= insta['DAYS_ENTRY_PAYMENT'], 0, 1)

In [41]:
insta['insta_debt'] = insta['AMT_INSTALMENT'] - insta['AMT_PAYMENT']

In [42]:
df2 = insta.sort_values('DAYS_INSTALMENT', ascending = False).groupby(['SK_ID_CURR']).agg({'DAYS_INSTALMENT':'first','insta_debt': ['sum', 'mean', 'first'], 'insta_delinquency' :['sum','first']})
df2.columns = df2.columns.map('_'.join)
df2

,DAYS_INSTALMENT_first,insta_debt_sum,insta_debt_mean,insta_debt_first,insta_delinquency_sum,insta_delinquency_first
SK_ID_CURR,,,,,,
100001,-1619.0,0.000,0.00000,0.0,1,0
100002,-25.0,0.000,0.00000,0.0,0,0
100003,-536.0,0.000,0.00000,0.0,0,0
100004,-724.0,0.000,0.00000,0.0,0,0
100005,-466.0,0.000,0.00000,0.0,1,0
...,...,...,...,...,...,...
456251,-30.0,0.000,0.00000,0.0,0,0
456252,-2316.0,0.000,0.00000,0.0,1,0
456253,-1716.0,3973.095,283.79250,0.0,1,0


In [43]:
pc = pd.read_csv('POS_CASH_balance.csv')
pc

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,1000001,158271,-8,2.0,0.0,Completed,0,0
1,1000004,260094,-26,10.0,7.0,Active,0,0
2,1000005,176456,-49,10.0,3.0,Active,0,0
3,1000008,152059,-40,10.0,7.0,Active,0,0
4,1000008,152059,-38,10.0,5.0,Active,0,0
...,...,...,...,...,...,...,...,...
1048570,2843494,292375,-24,2.0,0.0,Completed,0,0
1048571,2843497,451578,-15,24.0,18.0,Active,0,0
1048572,2843497,451578,-8,24.0,11.0,Active,0,0
1048573,2843499,314148,-35,60.0,55.0,Active,0,0


In [44]:
# flatten latest POS data for SK_ID_PREV
pc = pc.sort_values('MONTHS_BALANCE', ascending = False)

In [45]:
cols = ['MONTHS_BALANCE','SK_DPD','SK_DPD_DEF']
pos0 = pc.groupby(['SK_ID_CURR'])[cols].first()
pos1 = pc.groupby(['SK_ID_CURR'])[cols].nth(1)
pos2 = pc.groupby(['SK_ID_CURR'])[cols].nth(2)
pos3 = pc.groupby(['SK_ID_CURR'])[cols].nth(3)

In [46]:
from functools import reduce 
data_frames = [pos0, pos1, pos2, pos3]
poslag = reduce(lambda  left,right: pd.concat( [left,right], axis=1, sort= False), data_frames)
columns =[]
for i in range(4):
    columns += ['MONTHS_BALANCE' + str(i), 'SK_DPD'+ str(i),'SK_DPD_DEF'+ str(i) ]
poslag.columns = columns
poslag

,MONTHS_BALANCE0,SK_DPD0,SK_DPD_DEF0,MONTHS_BALANCE1,SK_DPD1,SK_DPD_DEF1,MONTHS_BALANCE2,SK_DPD2,SK_DPD_DEF2,MONTHS_BALANCE3,SK_DPD3,SK_DPD_DEF3
SK_ID_CURR,,,,,,,,,,,,
100002,-13,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100003,-23,0,0,-66.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
100005,-21,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100007,-3,0,0,-9.0,0.0,0.0,-10.0,0.0,0.0,-35.0,0.0,0.0
100008,-13,0,0,-17.0,0.0,0.0,-19.0,0.0,0.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
456248,-4,0,0,-10.0,0.0,0.0,-11.0,0.0,0.0,-16.0,0.0,0.0
456249,-40,0,0,-41.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
456250,-2,0,0,-13.0,0.0,0.0,-17.0,0.0,0.0,-18.0,0.0,0.0


In [47]:
insta_pos = pd.merge(df2, poslag, on='SK_ID_CURR', how= 'outer')
insta_pos

,DAYS_INSTALMENT_first,insta_debt_sum,insta_debt_mean,insta_debt_first,insta_delinquency_sum,insta_delinquency_first,MONTHS_BALANCE0,SK_DPD0,SK_DPD_DEF0,MONTHS_BALANCE1,SK_DPD1,SK_DPD_DEF1,MONTHS_BALANCE2,SK_DPD2,SK_DPD_DEF2,MONTHS_BALANCE3,SK_DPD3,SK_DPD_DEF3
SK_ID_CURR,,,,,,,,,,,,,,,,,,
100001,-1619.0,0.0,0.0,0.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100002,-25.0,0.0,0.0,0.0,0.0,0.0,-13.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100003,-536.0,0.0,0.0,0.0,0.0,0.0,-23.0,0.0,0.0,-66.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
100004,-724.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100005,-466.0,0.0,0.0,0.0,1.0,0.0,-21.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
447874,NaN,NaN,NaN,NaN,NaN,NaN,-92.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
450344,NaN,NaN,NaN,NaN,NaN,NaN,-41.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
452304,NaN,NaN,NaN,NaN,NaN,NaN,-7.0,0.0,0.0,-11.0,0.0,0.0,-11.0,0.0,0.0,-12.0,0.0,0.0


In [48]:
insta_pos.to_csv('insta_pos.csv')

In [49]:
prvs_app = pd.read_csv('prvs_app.csv')

In [50]:
prvs_app  = prvs_app.merge(insta_pos, on='SK_ID_CURR', how='outer')

In [52]:
prvs_app

,SK_ID_CURR,ratio_approved,ratio_refused,ratio_canceled,ratio_unused,DAYS_DECISION,NAME_CONTRACT_STATUS,AMT_CREDIT_x,NAME_CONTRACT_TYPE,AMT_APPLICATION,...,SK_DPD_DEF0,MONTHS_BALANCE1_y,SK_DPD1,SK_DPD_DEF1,MONTHS_BALANCE2_y,SK_DPD2,SK_DPD_DEF2,MONTHS_BALANCE3_y,SK_DPD3,SK_DPD_DEF3
0,100001,1.0,0.0,0.0,0.0,-1740.0,Approved,23787.0,Consumer loans,24835.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,100002,1.0,0.0,0.0,0.0,-606.0,Approved,179055.0,Consumer loans,179055.0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,100003,1.0,0.0,0.0,0.0,-746.0,Approved,1035882.0,Cash loans,900000.0,...,0.0,-66.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
3,100004,1.0,0.0,0.0,0.0,-815.0,Approved,20106.0,Consumer loans,24282.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,100005,0.5,0.0,0.5,0.0,-315.0,Canceled,0.0,Consumer loans,44617.5,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
340751,411869,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,-40.0,0.0,0.0,-49.0,0.0,0.0,NaN,NaN,NaN
340752,412695,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
340753,434453,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,-87.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
340754,443475,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [53]:
cols = prvs_app.select_dtypes(include = object).columns
prvs_app[cols] = prvs_app[cols].fillna('Missing')
cols = prvs_app.select_dtypes(exclude = object).columns
prvs_app[cols] = prvs_app[cols].fillna(0)
prvs_app.isnull().sum()

SK_ID_CURR                   0
ratio_approved               0
ratio_refused                0
ratio_canceled               0
ratio_unused                 0
DAYS_DECISION                0
NAME_CONTRACT_STATUS         0
AMT_CREDIT_x                 0
NAME_CONTRACT_TYPE           0
AMT_APPLICATION              0
AMT_CREDIT_y                 0
AMT_DOWN_PAYMENT             0
NAME_YIELD_GROUP             0
NFLAG_INSURED_ON_APPROVAL    0
RATE_INTEREST_PRIMARY        0
RATE_INTEREST_PRIVILEGED     0
Diff_applied_apprved         0
Days_consumerloans           0
AMT_credit_consumerloans     0
Days_cashloans               0
AMT_credit_cashloans         0
Days_revolvingloans          0
AMT_credit_revolvingloans    0
MONTHS_BALANCE0_x            0
balance_pct_change0          0
MONTHS_BALANCE1_x            0
balance_pct_change1          0
MONTHS_BALANCE2_x            0
balance_pct_change2          0
MONTHS_BALANCE3_x            0
balance_pct_change3          0
has_cc_now                   0
balance_

In [55]:
prvs_app.to_csv('prv_home.csv', index= False)